# Modeling

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import plotly.express as px
pd.options.display.max_rows = 5000
pd.options.display.max_columns = 5000

### Reading in salary and abbreviation data

In [2]:
sal = pd.read_csv('all_seasons_salaries.csv')
sal = sal[(sal['Team'].isna()!=True) & (sal['Salary'].isna()!=True)]
abr = pd.read_csv('nba_teams_abrev.csv')

d = dict(zip(abr['Team_Name'],abr['Tm']))
sal['Team'].replace('Seattle SuperSonics','Seattle Supersonics',inplace=True)
sal['Season'] = sal['Season'].apply(lambda x: x-100 if x>2020 else x)

tm = []
for i in sal['Team']:
    tm.append(d.get(i))
sal['Tm'] = tm
sal['Player_ID2'] = sal['Player_ID'] + "" +  sal['Tm'] + "" +sal['Season'].map(str)
sal.drop(columns='Unnamed: 0',inplace=True)

### Reading in and cleaning original data

In [3]:
df = pd.read_csv('df_for_model.csv')
X = df.groupby(['Player_ID','Player_Name']).count().reset_index()[['Player_ID','Player_Name']]
name_id_dict = dict(zip(X['Player_ID'],X['Player_Name']))
df1 = df.drop(columns=['Unnamed: 0', 'Unnamed: 0_x','Ht','Feet', 'Inches','Ht_Inches', 'Ht_m','Unnamed: 0_y','Player_Name_y','Rk_x',
       'Pos_y','Unnamed: 29','Unnamed: 0.1','Player_Name_x','Rk_y', 'Pos', 'Age_y',
       'Tm_y', 'G_y', 'MP_y','Unnamed: 19','Unnamed: 24','From','To','Player_Name_x','Player_Name','Pos_x','Birth Date','Colleges'],axis=1)
dt = df1.groupby(['Season','Tm_x','Player_ID']).mean()
dt.fillna(0)
dt.reset_index(inplace=True)
names = []
for i in dt['Player_ID']:
    names.append(name_id_dict[i])
dt['Player_Name'] = names

### Data Ready for Testing

In [4]:
def pca(dt):
    i = 2010
    dt['Player_ID2'] = dt['Player_ID'] + "" +  dt['Tm_x'] + "" +dt['Season'].map(str)
    cols = dt.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    dt = dt[cols]

    y = dt.groupby(['Player_ID2','Player_Name','Tm_x']).mean().reset_index()[['Player_ID2','Player_Name','Season','Tm_x']]
    id_name_dict = dict(zip(y['Player_ID2'],zip(y['Player_Name'],y['Season'],y['Tm_x'])))
    
    while i < 2020:
        dff = dt[(dt['Season']==i) & (dt['G_x']>20)]
        dff = dff.set_index('Player_ID2').iloc[:,4:].drop(columns='Age_x')   
        df_pca = StandardScaler().fit_transform(dff)
        pca = PCA()
        pca.set_params(n_components=17)
        df_pca = pca.fit_transform(df_pca)
        print(pca.explained_variance_ratio_)
        kmeans = KMeans()
        kmeans.set_params(n_clusters = 13)
        kmeans.fit(df_pca)
        # graph clusters
#         plt.figure(figsize=(12,8))
#         plt.scatter(df_pca[:, 0], df_pca[:, 1], c=kmeans.labels_, alpha=0.8)
#         n=6
#         # plt.text(df_pca[n,0], df_pca[n,1], dt.index.tolist()[n])
#         # plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], marker='+', s=1000, c=[0, 1])
#         plt.show()
        
        # creating df with cluster labels
        x = pd.DataFrame(df_pca,dff.index).reset_index()
        names = []
        for s in x['Player_ID2']:
            names.append(id_name_dict.get(s)[0])
        x['Player_Name'] = names
        x['Cluster'] = kmeans.labels_
        x['Year'] = i
        teams = []
        for t in x['Player_ID2']:
            teams.append(id_name_dict.get(t)[2])
        x['Team'] = teams
        x.to_csv(f'Cluster_Labels/{i}_cluster_labels.csv')
        i += 1

### Running PCA for each year and showing explained variance ratio for each component

In [5]:
pca(dt)

[0.29327576 0.25884801 0.09853267 0.06581838 0.05274434 0.03917663
 0.02890752 0.02729387 0.01981233 0.01870089 0.01538812 0.01332664
 0.01195179 0.01072172 0.00704424 0.00668195 0.00550232]
[0.27804409 0.26035442 0.10741951 0.07529258 0.05029076 0.03308218
 0.03070817 0.02589634 0.02048018 0.01733451 0.01606609 0.01420227
 0.01196231 0.01119867 0.00852625 0.00624548 0.00600281]
[0.29209198 0.25451018 0.09520474 0.0741046  0.04869601 0.03827943
 0.02901977 0.02844198 0.02156554 0.01829635 0.01449418 0.01402232
 0.01134425 0.01061943 0.00874588 0.00743844 0.00663481]
[0.29459857 0.24578948 0.09970998 0.07533664 0.05165564 0.03914084
 0.03114845 0.02576645 0.02092536 0.0170125  0.01471271 0.01336787
 0.01124647 0.01039861 0.0089677  0.00685672 0.00645967]
[0.29805868 0.25705673 0.09119424 0.07320331 0.04431839 0.03716479
 0.02975838 0.02812086 0.02131575 0.01983733 0.01569493 0.01406064
 0.01256079 0.01048798 0.00806074 0.00698383 0.00585077]
[0.30640707 0.22926812 0.10037117 0.07763037 

## Concatening Year Cluster DFs

In [6]:
def concatenate(indir='/Users/michaelrazumovsky/Desktop/NBA_WebScraping_Project_3/Cluster_Labels',
                outfile = '/Users/michaelrazumovsky/Desktop/NBA_WebScraping_Project_3/Cluster_Labels/Concatenated.csv'):
    os.chdir(indir)
    fileList = glob.glob('*.csv')
    dfList = []
    colnames = ['Index','Player_ID2','0','1','2','3','4','5','6','7','8','9','10','11','12','13',
                '14','15','16','Player_Name','Cluster','Year','Team']
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename,header=None,index_col=False)
        df.columns = colnames
        dfList.append(df)
    concatDf = pd.concat(dfList,axis=0)
    concatDf.columns = colnames
    concatDf.to_csv(outfile,index=None)

In [7]:
concatenate()

2012_cluster_labels.csv
2011_cluster_labels.csv
2017_cluster_labels.csv
2014_cluster_labels.csv
2019_cluster_labels.csv
2015_cluster_labels.csv
2018_cluster_labels.csv
2016_cluster_labels.csv
2010_cluster_labels.csv
2013_cluster_labels.csv


### Cleaning the concatenated file

In [8]:
cl = pd.read_csv('Concatenated.csv')
cl = cl[cl['Player_Name']!='Player_Name'].drop(columns='Index')
cl.to_csv('Clean_Clusters.csv')
cl.head()

,Player_ID2,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,Player_Name,Cluster,Year,Team
1,collija04ATL2012,-3.096963,-6.615518,-0.232077,-0.226708,0.740342,-1.989610,0.932527,-0.290904,-0.225257,1.197405,0.883080,2.038786,1.121495,0.756096,0.353940,0.353281,0.393489,Jason Collins,0,2012,ATL
2,greenwi01ATL2012,-2.059770,2.453583,-1.199364,-2.541157,0.826965,1.170296,-0.789176,-0.755535,0.341054,-0.173517,-1.077244,0.806585,-0.665500,0.746474,0.418729,-0.138262,-0.293853,Willie Green,8,2012,ATL
3,hinriki01ATL2012,-4.008677,0.365781,-2.518004,0.812851,0.033300,-0.598221,-0.848485,-0.311363,0.695337,0.078731,-0.542962,0.738282,0.168757,0.113710,-0.123632,0.093105,-0.601914,Kirk Hinrich,9,2012,ATL
4,johnsiv01ATL2012,2.612674,-1.730152,1.136174,1.397999,0.087299,1.442396,0.411097,-0.292430,-0.620541,-1.682722,-1.787137,1.127974,-1.278598,0.035211,0.910055,-0.309389,-0.171873,Ivan Johnson,2,2012,ATL
5,johnsjo02ATL2012,1.234045,5.951142,-1.093636,-1.694684,-0.750545,-1.191768,-0.423149,0.324486,0.290835,0.055894,0.343343,0.508890,1.093512,0.451479,-0.244063,-0.727626,0.431873,Joe Johnson,1,2012,ATL


## Cluster Labels and Salary Merged With Original Data

In [9]:
z = dt.groupby(['Player_ID2','Player_Name','Tm_x']).mean().reset_index()[['Player_ID2','Player_Name','Season','Tm_x']]
dict_merge = dict(zip(z['Player_ID2'],zip(z['Player_Name'],z['Season'],z['Tm_x'])))

In [10]:
dt = pd.merge(cl[['Player_ID2','0','1','Cluster']],dt,how='left',on='Player_ID2')
cols = dt.columns.tolist()
cols = cols[-1:] + cols[:-1]
dt = dt[cols]

In [14]:
df_sal = pd.merge(dt,sal[['Player_ID2','Salary']],how='left',on='Player_ID2')

In [15]:
df_sal['Salary'] = df_sal['Salary'].astype(str).map(lambda x: x.replace('$','').replace(',',''))
df_sal['Salary'].replace('< Minimum','507500',inplace=True)

In [18]:
df_sal['Salary'] = df_sal['Salary'].astype(float)

In [22]:
df_sal.to_csv('df_wsal_cluster.csv')

In [23]:
df_sal.head()

,Player_Name,Player_ID2,0,1,Cluster,Season,Tm_x,Player_ID,Wt,Ht_cm,Age_x,G_x,GS,MP_x,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Salary
0,Jason Collins,collija04ATL2012,-3.096963,-6.615518,0,2012,ATL,collija04,255.0,213.0,33,30,10.0,308,2.8,6.9,0.400,0.0,0.0,0.000,2.8,6.9,0.400,1.2,2.6,0.467,2.1,6.2,8.3,1.7,0.7,0.5,1.7,5.9,6.7,87.0,104.0,3.7,0.418,0.000,0.375,4.6,13.6,9.1,4.6,0.7,0.7,17.7,8.6,-0.2,0.3,0.2,0.025,-5.6,0.5,-5.1,-0.2,1352181.0
1,Willie Green,greenwi01ATL2012,-2.059770,2.453583,8,2012,ATL,greenwi01,200.0,193.0,30,53,2.0,922,8.8,18.8,0.471,2.9,6.5,0.442,5.9,12.2,0.486,2.8,3.2,0.857,0.9,3.8,4.6,2.3,1.3,0.3,1.9,4.3,23.3,110.0,105.0,13.9,0.578,0.348,0.172,1.9,8.2,5.1,7.4,1.3,0.5,8.6,19.5,1.4,0.9,2.4,0.122,0.3,-2.0,-1.7,0.1,1223166.0
2,Kirk Hinrich,hinriki01ATL2012,-4.008677,0.365781,9,2012,ATL,hinriki01,190.0,193.0,31,48,31.0,1237,5.2,12.7,0.414,2.0,5.8,0.346,3.2,6.8,0.472,1.1,1.4,0.781,0.5,3.9,4.3,5.7,1.6,0.5,2.5,4.3,13.6,100.0,104.0,9.2,0.511,0.461,0.108,1.1,8.5,4.8,16.3,1.6,0.7,15.8,13.9,0.4,1.4,1.8,0.069,-1.4,-0.1,-1.4,0.2,8100000.0
3,Ivan Johnson,johnsiv01ATL2012,2.612674,-1.730152,2,2012,ATL,johnsiv01,230.0,203.0,27,56,0.0,934,8.0,15.7,0.513,0.1,0.3,0.333,7.9,15.3,0.517,4.1,5.7,0.720,3.7,9.2,12.9,1.8,2.4,1.0,3.9,6.6,20.3,101.0,98.0,14.3,0.558,0.022,0.364,8.2,20.1,14.2,5.7,2.4,1.5,17.8,19.5,0.5,1.7,2.2,0.113,-2.9,1.0,-1.9,0.0,473604.0
4,Joe Johnson,johnsjo02ATL2012,1.234045,5.951142,1,2012,ATL,johnsjo02,240.0,201.0,30,60,60.0,2127,10.6,23.3,0.454,3.1,8.1,0.388,7.5,15.2,0.489,4.0,4.7,0.849,1.3,4.2,5.5,5.8,1.2,0.3,2.9,1.9,28.2,110.0,105.0,18.4,0.557,0.346,0.200,2.9,9.2,6.1,19.7,1.2,0.5,10.3,24.9,4.4,2.1,6.4,0.145,3.4,-1.4,2.1,2.2,18038573.0


In [ ]:
df.loc[df['First Season'] > 1990, 'First Season'] = 1

In [29]:
sal.sort_values(by='Season').head()

,Season,Team,Lg,Salary,Player_ID,Tm,Player_ID2
13708,1985,New York Knicks,NBA,"$230,000",walkeda01,NYK,walkeda01NYK1985
13590,1985,Dallas Mavericks,NBA,"$250,000",vinceja01,DAL,vinceja01DAL1985
13572,1985,Kansas City Kings,NBA,"$100,000",verhope01,KCK,verhope01KCK1985
8030,1985,Detroit Pistons,NBA,"$330,000",longjo01,DET,longjo01DET1985
11381,1985,San Antonio Spurs,NBA,"$86,000",roberfr01,SAS,roberfr01SAS1985


In [30]:
sal.Season.unique()

array([1991, 1992, 1993, 1994, 1995, 1985, 1986, 1988, 1989, 1996, 1997,
       1998, 2001, 1999, 2000, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2010, 2017, 2018, 2019, 2009, 2013, 2014, 2015, 2016, 2011, 2012,
       1987, 1990])